In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('data.csv')
df.head(10)

,english,spanish
0,Go.,Ve.
1,Go.,Vete.
2,Go.,Vaya.
3,Go.,Váyase.
4,Hi.,Hola.
5,Run!,¡Corre!
6,Run.,Corred.
7,Who?,¿Quién?
8,Fire!,¡Fuego!
9,Fire!,¡Incendio!


In [4]:
df.columns

Index(['english', 'spanish'], dtype='object')

In [5]:
source_vocab = set(' '.join(df['english'][25:125]))
target_vocab = set(' '.join(df['spanish'][25:125]))
source_vocab_size = len(source_vocab)
target_vocab_size = len(target_vocab)

In [6]:
print(source_vocab)
print(target_vocab)

{'u', 'q', 'H', 'D', 'O', ' ', 'N', 'n', 'A', 'C', 'B', 'S', 'g', 'L', 'r', 'p', 'f', "'", 'c', '1', 't', '9', 'W', 'l', 'a', 's', 'x', 'i', 'I', 'y', 'w', 'G', 'e', 'b', 'k', 'h', 'T', '.', 'm', 'o', '?', 'd', 'R', '!'}
{'q', 'u', 'H', 'D', 'O', 'j', 'U', ' ', 'N', 'V', 'n', 'C', 'A', 'E', 'É', 'B', 'S', 'g', 'é', 'L', 'r', 'ú', ',', 'p', 'f', 'c', 'Y', '¿', 't', 'R', 'í', 'M', '!', 'z', 'l', 's', 'i', 'á', 'I', 'y', 'v', 'Ó', 'G', 'e', 'b', 'ó', 'h', 'T', '¡', '.', 'P', 'Á', 'm', 'o', '?', 'a', 'd'}


In [7]:
source_char_to_int = {char: idx for idx, char in enumerate(source_vocab)}
target_char_to_int = {char: idx for idx, char in enumerate(target_vocab)}
source_int_to_char = {idx: char for char, idx in source_char_to_int.items()}
target_int_to_char = {idx: char for char, idx in target_char_to_int.items()}

In [8]:
# Convert text sequences to integer sequences
source_sequences = [[source_char_to_int[char] for char in text] for text in df['english'][25:125]]
target_sequences = [[target_char_to_int[char] for char in text] for text in df['spanish'][25:125]]

In [9]:
# Pad sequences to the same length
max_sequence_length = max(len(seq) for seq in source_sequences)
source_sequences = tf.keras.preprocessing.sequence.pad_sequences(source_sequences, maxlen=max_sequence_length, padding='post')
target_sequences = tf.keras.preprocessing.sequence.pad_sequences(target_sequences, maxlen=max_sequence_length, padding='post')

In [10]:
# Build the model
input_shape = (max_sequence_length, source_vocab_size)
output_shape = (max_sequence_length, target_vocab_size)

model = tf.keras.models.Sequential([
    # Embedding layer with a larger dimension for richer word representations
    tf.keras.layers.Embedding(source_vocab_size, 512, input_length=max_sequence_length),

    # First Bidirectional RNN layer with dropout and L2 regularization
    tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(512, return_sequences=True, 
                                kernel_regularizer=tf.keras.regularizers.l2(0.001))),
    tf.keras.layers.Dropout(0.4),

    # Second RNN layer with higher units and dropout
    tf.keras.layers.SimpleRNN(512, return_sequences=True),
    tf.keras.layers.Dropout(0.4),

    # Third RNN layer for more complex patterns
    tf.keras.layers.SimpleRNN(256, return_sequences=True),
    tf.keras.layers.Dropout(0.3),

    # Dense layer for more feature learning
    tf.keras.layers.Dense(512, activation='relu'),

    # Output layer to map to the target vocabulary
    tf.keras.layers.Dense(target_vocab_size, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
# One-hot encode the target sequences
target_sequences_one_hot = np.array([tf.keras.utils.to_categorical(seq, num_classes=target_vocab_size) for seq in target_sequences])

In [12]:
# Train the model
model.fit(source_sequences, target_sequences_one_hot,batch_size = 64, epochs=250)

Epoch 1/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 13s 73ms/step - accuracy: 0.0383 - loss: 5.0589
Epoch 2/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.1816 - loss: 4.4064
Epoch 3/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.2055 - loss: 4.0004
Epoch 4/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.2451 - loss: 3.7294
Epoch 5/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.2933 - loss: 3.5840
Epoch 6/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.3071 - loss: 3.4270
Epoch 7/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.3330 - loss: 3.2929
Epoch 8/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.3658 - loss: 3.1366
Epoch 9/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.3885 - loss: 3.0769
Epoch 10/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.3884 - loss: 2.9481
Epoch 11/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.4352 - loss: 2.8165
Epoch 12/250
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.4594 - l

In [13]:
# Translate a new input sequence
x = input("Enter String : ")
input_sequence = x
input_sequence = [source_char_to_int[char] for char in input_sequence]
input_sequence = tf.keras.preprocessing.sequence.pad_sequences([input_sequence], maxlen=max_sequence_length, padding='post')
output_sequence = model.predict(input_sequence)[0]
# Decode the output sequence
output_sequence = [target_int_to_char[np.argmax(char)] for char in output_sequence]
print("Input Sequence:",x)
print("Translated Sequence:", ''.join(output_sequence))

Enter String :  Thanks


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Input Sequence: Thanks
Translated Sequence: Gracias!
